# HyperOpt for CYP2D6 dataset
I used cyp2d6_union_trainset_base_smiles.csv from MoDaC old data.
Goal:Generate models with RF, NN, XGBoost with different parameter setting and select best model based on valid_r2 score.

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, time
import multiprocessing
from joblib import Parallel, delayed

from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse
from atomsci.ddm.pipeline import predict_from_model as pfm 
from atomsci.ddm.pipeline import compare_models as cm 

sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")
pal = sns.color_palette()

pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 150)

print("Imports Done.")

Imports Done.


In [20]:
target='CYP2D6'

# load data and examine

In [21]:
data_dir = '/mnt/projects/ATOM/sarkart4/Data/'+target+'-ampl-1.1.0_old/'
outdir = '/mnt/projects/ATOM/sarkart4/Data/HPO_models_'+target+'_old/'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [22]:
#test_file = data_dir + "cyp2d6_union_testset_base_smiles.csv"
data_file = data_dir + target.lower()+"_union_trainset_base_smiles.csv"
data = pd.read_csv(data_file)
data.head(2)

,base_rdkit_smiles,compound_id,pIC50,relation,active
0,COCCOc1ccn2c(-c3ccc4cccc(OC5CCNCC5)c4n3)cnc2c1,CHEMBL3109340,5.00000,NaN,1
1,CN1CCC[C@@H](CC2c3ccccc3Sc3ccccc32)C1,MJFJKKXQDNNUJF-HNNXBMFYNA-N,6.50004,NaN,1


In [23]:
print("Duplicate SMILES:",data.duplicated(subset="base_rdkit_smiles").sum(), "\tDuplicate compound IDs:",data.duplicated(subset="compound_id").sum(), "\tDuplicate indices:",data.index.duplicated().sum(), "\tNA SMILES:",data.base_rdkit_smiles.isna().sum())
print("Shape:", data.shape)
print('CYP2D6 actives:', data.active.sum())
print('CYP2D6 inactives:', len(data)-data.active.sum())

Duplicate SMILES: 0 	Duplicate compound IDs: 0 	Duplicate indices: 0 	NA SMILES: 0
Shape: (10445, 5)
CYP2D6 actives: 6982
CYP2D6 inactives: 3463


In [24]:
## save split uuids for future reference
# all are mordread_filtered
# /home/sarkart4/MoDaC/NCI_DOE_Archive/ATOM/Safety_Screen_Targets/CYP2D6-ampl-1.1.0/cyp2d6_union_trainset_base_smiles_train_valid_test_scaffold_c8c36365-b294-41a4-983a-57879bbde0c0.csv
# /home/sarkart4/MoDaC/NCI_DOE_Archive/ATOM/Safety_Screen_Targets/CYP2D6-ampl-1.1.0/cyp2d6_union_trainset_base_smiles_train_valid_test_scaffold_4cb49d5e-1882-474e-ae0b-781d77a4738a.csv
# Dataset split table saved to /mnt/projects/ATOM/sarkart4/Data/CYP2D6-ampl-1.1.0/cyp2d6_union_trainset_base_smiles_train_valid_test_scaffold_5fdd8051-5b67-4a13-a768-a8a702d0182b.csv

#CYP2D6_old FRCE
#Dataset split table saved to /mnt/projects/ATOM/sarkart4/Data/CYP2D6-ampl-1.1.0_old/cyp2d6_union_trainset_base_smiles_train_valid_test_scaffold_c391ef72-9cd5-4bb7-a88e-9c55be5012a9.csv
#split_uuid = 'c391ef72-9cd5-4bb7-a88e-9c55be5012a9' # scaffold, ecfp
split_uuid =  '4dc24a89-e3c9-4c91-b8b1-6244f5a76132'# scaffold, graphconv
#split_uuid =  '621c5873-bd9e-400e-a379-fd94996d547a' # scaffold, rdkit_raw
#split_uuid = 'c517f69d-dd98-46c9-a9ae-94e6488149fb' # scaffold,mord
[split_uuid,data_file]

['4dc24a89-e3c9-4c91-b8b1-6244f5a76132',
 '/mnt/projects/ATOM/sarkart4/Data/CYP2D6-ampl-1.1.0_old/cyp2d6_union_trainset_base_smiles.csv']

In [ ]:
#mf = pd.read_csv(data_dir+"scaled_descriptors/cyp2d6_union_trainset_base_smiles_with_mordred_filtered_descriptors.csv")
#mf.shape

# Grid Search - RF

In [13]:
params = {
  #  "hyperparam": "True",
    "search_type" : "user_specified",
    "save_results": "False",
    "collection_name": "CYP2D6",
    "rerun": "False",
    "dataset_key" : data_file,
    "datastore": "False",
    "response_cols" : "pIC50", 
    "id_col": "compound_id",
    "smiles_col" : "base_rdkit_smiles",
    "result_dir": outdir, 
    "split_only": "False",
    "previously_split": "True",
    "split_uuid": split_uuid,
    "splitter": "scaffold",
    #"split_valid_frac": "0.15",
    #"split_test_frac": "0.002",
    "prediction_type": "regression",
    "transformers": "True",
    "featurizer": "computed_descriptors",
    "descriptor_type": "rdkit_raw", 
    "max_epochs": "100",
    "model_type" :"RF",
    "uncertainty": "True",
    "verbose": "True"
}

In [14]:
# # 210 models per feature set
# estimator_choice = [512]
# depth_choice = [128]
# features_choice = [64] #16,24,32,64,128, # can only be < total number of features

estimator_choice = [16,24,32,64,128,256,512]
depth_choice = [16,24,32,64,128,256]
features_choice = [16,24,32,64,128] #16,24,32,64,128, # can only be < total number of features

for rf_estimator in estimator_choice:
  for rf_depth in depth_choice:
    for rf_max_feature in features_choice:
        params["rf_estimators"] = rf_estimator
        params["rf_max_depth"] = rf_depth
        params["rf_max_features"] = rf_max_feature
        tp = parse.wrapper(params)
        pl = mp.ModelPipeline(tp)
        pl.train_model()
        pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
        pred_results = pred_data.get_prediction_results()
        print(f"rf_estimators: {rf_estimator}, rf_max_depth: {rf_depth}, rf_max_features: {rf_max_feature}, valid_r2: {pred_results['r2_score']}")

INFO:ATOM:Created a dataset hash 'a0eb2e6092b3542373f67528e299e283' from dataset_key '/mnt/projects/ATOM/sarkart4/Data/CYP2D6-ampl-1.1.0_old/cyp2d6_union_trainset_base_smiles.csv'
DEBUG:ATOM:Attempting to load featurized dataset
DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/RF_computed_descriptors_scaffold_regression/dbfc8da7-0711-4811-923a-08245daa1bb6/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in true_divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Transformi

rf_estimators: 16, rf_max_depth: 16, rf_max_features: 16, valid_r2: 0.16726828648589243


DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/RF_computed_descriptors_scaffold_regression/9d0c9672-2d33-48f6-83fb-29f1244c1289/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in true_divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Fitting random forest model
INFO:ATOM:Fold 0: training r2_score = 0.797, validation r2_score = 0.174, test r2_score = 0.051
INFO:ATOM:Wrote model tarball to /mnt/proj

rf_estimators: 16, rf_max_depth: 16, rf_max_features: 24, valid_r2: 0.17367151791778357


DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/RF_computed_descriptors_scaffold_regression/a5b28328-4dee-43c7-bc6f-b3827c4f3589/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in true_divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Fitting random forest model
INFO:ATOM:Fold 0: training r2_score = 0.816, validation r2_score = 0.156, test r2_score = -0.093
INFO:ATOM:Wrote model tarball to /mnt/pro

...

rf_estimators: 512, rf_max_depth: 256, rf_max_features: 32, valid_r2: 0.21426767640283895


DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/RF_computed_descriptors_scaffold_regression/ded4095e-2e9e-42e4-a6e3-6e68fc33a6f5/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in true_divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Fitting random forest model
INFO:ATOM:Fold 0: training r2_score = 0.922, validation r2_score = 0.217, test r2_score = -0.145
INFO:ATOM:Wrote model tarball to /mnt/pro

rf_estimators: 512, rf_max_depth: 256, rf_max_features: 64, valid_r2: 0.21678202451589546


DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/RF_computed_descriptors_scaffold_regression/583bac7a-b8b4-4450-95df-957374523f7f/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in true_divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
INFO:ATOM:Fitting random forest model
INFO:ATOM:Fold 0: training r2_score = 0.922, validation r2_score = 0.210, test r2_score = -0.087
INFO:ATOM:Wrote model tarball to /mnt/pro

rf_estimators: 512, rf_max_depth: 256, rf_max_features: 128, valid_r2: 0.20972982758172243


# Grid Search - RF (parallel)

In [ ]:
multiprocessing.cpu_count()

In [ ]:
## 210 models per feature set
estimator_choice = [16,24,32,64,128,256,512]
depth_choice = [16,24,32,64,128,256]
features_choice = [16,24,32,64,128] #16,24,32,64,128, # can only be < total number of features

param_combos = [] # list of params dict: each element is a params dict with specific set of leyrs,dropouts, lr_rate combinations
for rf_estimator in estimator_choice:
  for rf_depth in depth_choice:
    for rf_max_feature in features_choice:
        params["rf_estimators"] = rf_estimator
        params["rf_max_depth"] = rf_depth
        params["rf_max_features"] = rf_max_feature
        param_combos.append(params)
print(" total models = ", len(param_combos))

def parallelrun(params_):
    tp = parse.wrapper(params_)
    pl = mp.ModelPipeline(tp)
    pl.train_model()
start=time.time()
cores = 4 #multiprocessing.cpu_count()
Parallel(n_jobs=cores, timeout=99999)(delayed(parallelrun)(params_)for params_ in param_combos)
print(" Time taken to create ", len(param_combos), "RF models(min)= ", np.round((time.time()-start)/60,2))

# Grid Search - NN

In [26]:
params={
 'collection_name': 'CYP2D6',
 'dataset_key': data_file,
 'datastore': 'False',
 "featurizer": "graphconv", #"computed_descriptors",
 #"descriptor_type": "rdkit_raw", #mordred_filtered",
 #'hyperparam': 'True',
 'id_col': 'compound_id',
 'model_type': 'NN',
 'prediction_type': 'regression',
 'previously_split': 'True',
 'split_uuid': split_uuid,
 'rerun': 'False',
 'response_cols': 'pIC50',
 'result_dir': outdir,
 'save_results': 'False',
 'search_type': 'user_specified',
 'smiles_col': 'base_rdkit_smiles',
 'split_only': 'False',
 #'split_test_frac': '0.002',
 #'split_valid_frac': '0.15',
 'splitter': 'scaffold',
 'transformers': 'True',
 'uncertainty': 'True',
 #"dropouts": "0.10,0.10,0.10",
 #"layer_sizes": "64,64,64",
 #"learning_rate": "0.000753",
 'verbose': 'True'}

In [ ]:
# (SLOW - more than 12 hours)
layer_dropout = [
#                 ('64,16','0.3,0.3'),
#                 ('64,16','0.5,0.5'),
                 #('128,32','0.0,0.0'),
                 ('128,32','0.5,0.5'),
                 #('256,64','0.0,0.0'),
                 ('256,64','0.5,0.5'),
                 #('512,128','0.0,0.0'),
                 ('512,128','0.5,0.5'),
                 #('256,64,16','0.0,0.0,0.0'),
                 ('256,64,16','0.5,0.5,0.5'),
                 #('512,256,32','0.0,0.0,0.0'),
                 ('512,256,32','0.5,0.5,0.5'),
                 #('512,256,128','0.0,0.0,0.0'),
                 ('512,256,128','0.5,0.5,0.5'),
                 ('512,128,128','0.5,0.5,0.5'),
                 ('512,256,128,64','0.5,0.5,0.5,0.5'),

]
lr_choice = [0.0007, 0.005] # [.00001,.00005,.0001,.0005,.001,.005,.01,.05]

for layers,dropouts in layer_dropout:
    for learning_rate in lr_choice:
        params["layer_sizes"] = layers
        params["dropouts"] = dropouts
        params["learning_rate"] = learning_rate
        tp = parse.wrapper(params)
        pl = mp.ModelPipeline(tp)
        pl.train_model()
        pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
        pred_results = pred_data.get_prediction_results()
        print(f"layers: {layers}, dropouts: {dropouts}, learning rate: {learning_rate}, valid_r2: {pred_results['r2_score']}\n")

INFO:ATOM:Created a dataset hash 'a0eb2e6092b3542373f67528e299e283' from dataset_key '/mnt/projects/ATOM/sarkart4/Data/CYP2D6-ampl-1.1.0_old/cyp2d6_union_trainset_base_smiles.csv'
DEBUG:ATOM:Attempting to load featurized dataset
DEBUG:ATOM:Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
INFO:ATOM:Featurized dataset not previously saved for dataset cyp2d6_union_trainset_base_smiles, creating new
INFO:ATOM:Featurizing sample 0
INFO:ATOM:Featurizing sample 1000
INFO:ATOM:Featurizing sample 2000
INFO:ATOM:Featurizing sample 3000
INFO:ATOM:Featurizing sample 4000
INFO:ATOM:Featurizing sample 5000
INFO:ATOM:Featurizing sample 6000
INFO:ATOM:Featurizing sample 7000
INFO:ATOM:Featurizing sample 8000
INFO:ATOM:Featurizing sample 9000
INFO:ATOM:Featurizing sample 10000


number of features: 75


INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/NN_graphconv_scaffold_regression/8cb766e2-04f6-4d8e-a607-94063802f8ef/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming response data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Reshape_14:0", shape=(177,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Reshape_13:0", shape=(177, 128), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % va

/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Reshape_26:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Reshape_25:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Cast_8:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_1/Reshape_29:0", shape=(None,), dtype=int32), values=Tens

layers: 128,32, dropouts: 0.5,0.5, learning rate: 0.0007, valid_r2: 0.07260191088157875



...

number of features: 75


INFO:ATOM:Wrote transformers to /mnt/projects/ATOM/sarkart4/Data/HPO_models_CYP2D6_old/cyp2d6_union_trainset_base_smiles/NN_graphconv_scaffold_regression/f559ad0c-a3b6-4195-a602-440e6ea855df/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming response data
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_14:0", shape=(179,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_13:0", shape=(179, 256), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory.

/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_14:0", shape=(218,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_13:0", shape=(218, 256), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_17:0", shape=(1504,), dtype=int32), value

/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_14:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_13:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_17:0", shape=(None,), dtype=int32), val

/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_26:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_25:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Cast_8:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_23/graph_pool_35/Reshape_29:0", shape=(None,), dtype=int32), val

# Grid Search - NN (submit batch jobs FRCE- Titli's code new)

In [ ]:
import multiprocessing
from joblib import parallel, delayed
multiprocessing.cpu_count()

In [ ]:
param_combos = [] # list of params dict: each element is a params dict with specific set of leyrs,dropouts, lr_rate combinations

# layer_dropout = [
#                 #  ('64,16','0.1,0.1'),
#                 #  ('64,16','0.2,0.2'),
#                  ('64,16','0.3,0.3'),
#                  ('64,16','0.4,0.4'),
#                  ('128,32','0.1,0.1'),
#                  ('128,32','0.2,0.2'),
#                  ('128,32','0.3,0.3'),
#                  ('128,32','0.4,0.4'),
#                  ('256,64','0.1,0.1'),
#                  ('256,64','0.2,0.2'),
#                  ('256,64','0.3,0.3'),
#                  ('256,64','0.4,0.4'),
#                  ('512,128','0.1,0.1'),
#                  ('512,128','0.2,0.2'),
#                  ('512,128','0.3,0.3'),
#                  ('512,128','0.4,0.4'),
#                  ('256,64,16','0.1,0.1,0.1'),
#                  ('256,64,16','0.2,0.2,0.2'),
#                  ('256,64,16','0.3,0.3,0.3'),
#                  ('256,64,16','0.4,0.4,0.4'),
# ]
# lr_choice = [.00001,.00005,.0001,.0005,.001,.005,.01,.025,.05,.075]
# def parallelrun(params):
#     tp = parse.wrapper(params)
#     pl = mp.ModelPipeline(tp)
#     pl.train_model()
#     pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
#     pred_results = pred_data.get_prediction_results()
#     print(f"layers: {layers}, dropouts: {dropouts}, learning rate: {learning_rate}, valid_r2: {pred_results['r2_score']}\n")
    
# #Parallel(n_jobs=multiprocessing.cpu_count(), verbose=50)delayed(parallelrun)(params)for combo in combos)
param_combos=[]
for layers,dropouts in layer_dropout:
  for learning_rate in lr_choice:
    params[]

# Grid Search - xGBoost

In [ ]:
params={
 'collection_name': 'CYP2D6',
 'dataset_key': data_file,
 'datastore': 'False',
 'featurizer': 'computed_descriptors',
 'descriptor_type':'mordred_filtered',
 'id_col': 'compound_id',
 'lc_account': 'None',
 'max_epochs': '100',
 'model_type': 'xgboost',
 'prediction_type': 'regression',
 'previously_split': 'True',
 'rerun': 'False',
 'response_cols': 'pIC50',
 'result_dir': outdir,
 'save_results': 'False',
 'smiles_col': 'base_rdkit_smiles',
 'split_uuid': split_uuid,
 'transformers': 'True',
 'uncertainty': 'True',
 'verbose': 'False'}

In [ ]:
# 96 models per feature set, ~40 seconds per model; ~1:04 hours
gamma_choice = [4,8,16] # done [0.00,0.05, 0.1,0.2,0.3,0.4,0.5,1,2]
lr_choice = [0.15,0.10,0.05,0.01,0.001,0.0001] # done  [0.25,0.2] 

for gamma in gamma_choice:
    for learning_rate in lr_choice:
        params["xgb_gamma"] = gamma
        params["xgb_learning_rate"] = learning_rate
        tp = parse.wrapper(params)
        pl = mp.ModelPipeline(tp)
        pl.train_model()
        pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
        pred_results = pred_data.get_prediction_results()
        print(f"xgb gamma: {gamma}, xgb learning rate: {learning_rate}, valid_r2: {pred_results['r2_score']}\n")

# Monitor HP search with *groupby*

In [ ]:
# source = outdir+'RF_GridSearch/*'
# ! mv $source $outdir
# !ls $outdir

In [ ]:
perf_df = cm.get_summary_perf_tables(collection_names=None, filter_dict={}, result_dir=outdir, prediction_type='regression', verbose=False)
#perf_df = perf_df[perf_df.rf_estimators!=500]
perf_df = perf_df.sort_values(by="valid_r2_score", ascending=False)
perf_df.head(2)

In [ ]:
perf_df.shape

In [ ]:
perf_df.groupby(by=['model_type', 'features']).count()[['model_uuid']].T

In [ ]:
rfmods = perf_df[perf_df.model_type=="RF"]
print(rfmods.shape)
print(rfmods.head(1)['valid_r2_score'])
rfmods.head(2)

In [ ]:
nnmods = perf_df[perf_df.model_type=="NN"]
print(nnmods.shape)
print(nnmods.head(1)['valid_r2_score'])
nnmods.head(2)

In [ ]:
xgboostmods = perf_df[perf_df.model_type=="xgboost"]
print(xgboostmods.shape)
print(xgboostmods.head(1)['valid_r2_score'])
xgboostmods.head(2)

In [ ]:
# should be 10 each, 20 groups per feature, if you got all the models created
nnmods.groupby(by=['features','layer_sizes','dropouts']).count()[['model_uuid']]

In [ ]:
#visual of RF HPO results
fig, ax = plt.subplots(1,3,figsize=(21,7))
sns.boxplot(data=perf_df, x="rf_estimators", y="valid_r2_score", ax=ax[0])
sns.boxplot(data=perf_df, x="rf_max_depth", y="valid_r2_score", ax=ax[1])
sns.boxplot(data=perf_df, x="rf_max_features", y="valid_r2_score", ax=ax[2])
fig.tight_layout()

# Baysean optimization NN - manually

In [ ]:
#define loss function
def lossfn(p):
    params["learning_rate"] = float(p["learn_rate"])
    params["layer_sizes"] = int(p["layer_size"])
    params["layer_nums"] = int(p["layer_num"])
    params["dropouts"] = float(p["dropout"])

    tparam = parse.wrapper(params)
    pl = mp.ModelPipeline(tparam)
    pl.train_model()
    perf_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
    pred_results = perf_data.get_prediction_results()
    r2 = pred_results['r2_score']
    return {'loss': 1-r2, 
            'status': STATUS_OK, 
            'model': tparam.model_tarball_path, 
            'model_uuid': tparam.model_uuid,
            'learn_rate': params["learning_rate"],
            'layer_size': params["layer_sizes"],
            'layer_num': params["layer_nums"],
            'dropout': params['dropouts']}

space = {
    "learn_rate": hp.uniform("learn_rate", 0.002, 0.006),
    "layer_size": hp.uniform("layer_size", 16, 256),
    "layer_num": hp.uniform("layer_num", 2, 4),
    "dropout": hp.uniform("droupout", 0.1, 0.6)

}

In [ ]:
trials = Trials()
best = fmin(lossfn, space, algo=tpe.suggest, max_evals=5, trials=trials)

In [ ]:
r2_list = [1-trials.trials[i]["result"]["loss"] for i in range(len(trials.trials))]
fig, ax = plt.subplots(1,1,figsize=(10,10))
sns.lineplot(x=list(range(1,len(trials.trials)+1)), y=r2_list, ax=ax)
#ax.set_ylim(0.2,0.6)

In [ ]:
perf_df = pd.DataFrame({"model_uuid": model_uuid_list,"learning_rate": lr_list, "layer_sizes": ls_list, "layer_nums": ln_list, "dropouts": drop_list, "valid_r2_score": r2_list})
perf_df.sort_values(by="valid_r2_score", ascending=False)
# perf_df.to_csv(file_dir + 'NN_Bayesian_Search/' + target_name + '_NN_Bayesian_Search_5_trials.csv', index=False)

In [ ]:
#Save Bayesian optimization trial object and continue later
# save the trial object
trial_file = os.path.join("hp_trial.pkl")
with open(trial_file, "wb") as f:
    pickle.dump(trials, f)



In [ ]:
# load the trial object
with open(trial_file, "rb") as f:
    new_trials = pickle.load(f)

In [ ]:
# continue HPO
best = fmin(lossfn, space, algo=tpe.suggest, max_evals=50, trials=new_trials)

In [ ]:
r2_list = [1-new_trials.trials[i]["result"]["loss"] for i in range(len(new_trials.trials))]
fig, ax = plt.subplots(1,1,figsize=(10,10))
sns.lineplot(x=list(range(1,len(new_trials.trials)+1)), y=r2_list, ax=ax)
#ax.set_ylim(0.2,0.6)

In [ ]:
lr_list = [trials.trials[i]["result"]["learn_rate"] for i in range(len(trials.trials))]
ls_list = [trials.trials[i]["result"]["layer_size"] for i in range(len(trials.trials))]
ln_list = [trials.trials[i]["result"]["layer_num"] for i in range(len(trials.trials))]
drop_list = [trials.trials[i]["result"]["dropout"] for i in range(len(trials.trials))]
model_uuid_list = [trials.trials[i]["result"]["model_uuid"] for i in range(len(trials.trials))] 

### RF

In [ ]:
! tail -qn 2 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_RF.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_RF_mordred_seltrain.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_txptr_hyperopt_RF.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_scr_hyperopt_RF.json.out

### NN computed descriptors

In [ ]:
! tail -qn 1 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_NN_3_noGC.json.out

In [ ]:
# ! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_NN_mordred_sel.json.out

In [ ]:
# ! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_NN_mordred_seltrain.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_txptr_hyperopt_NN.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_scr_hyperopt_NN.json.out

### NN GC

In [ ]:
! tail -qn 1 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_NN_3_GC.json.out

### XG boost

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_hyperopt_xg_mordred_seltrain.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_txptr_hyperopt_xg.json.out

In [ ]:
! tail -qn 3 /g/g16/apaulson/train_JSON_FILE.DILIst_scr_hyperopt_xg.json.out

# Best model

In [ ]:
import warnings
warnings.filterwarnings(action='once')
from atomsci.clients import MLMTClient
import atomsci.ddm.pipeline.model_tracker as mt
import atomsci.ddm.pipeline.compare_models as cmp
import pandas as pd

In [ ]:
client=MLMTClient()
for coll in client.get_collection_names()[2:19]:
    collections = [coll]
    res=client.count_models(coll)
    print(f'Num models in {coll}: {res}')

In [ ]:
collections=['DILIst_class']
responsecols = 'DILIst_Classification'
allmodels1=cmp.get_multitask_perf_from_tracker(collections[0], response_cols=responsecols, 
                                              expand_subsets='train,test,val', exhaustive=True)
allmodels1=allmodels1[allmodels1.prediction_type=='classification']
allmodels1['model_run']='No bio data'
allmodels1=allmodels1.reset_index(drop=True)
print(allmodels1.best_valid_roc_auc_score.max())
print(allmodels1.shape)
allmodels1.groupby(by=['model_type', 'features', 'best_valid_num_compounds']).count()[['model_uuid']].T

In [ ]:
collections=['DILIst_txptr_class']
responsecols = 'DILIst_Classification'
allmodels2=cmp.get_multitask_perf_from_tracker(collections[0], response_cols=responsecols, 
                                              expand_subsets='train,test,val', exhaustive=True)
allmodels2=allmodels2[allmodels2.features.str.contains('scr')]
allmodels2['model_run']='Scram bio data'
allmodels2=allmodels2.reset_index(drop=True)
print(allmodels2.best_valid_roc_auc_score.max())
print(allmodels2.shape)
allmodels2.groupby(by=['model_type', 'features', 'best_valid_num_compounds']).count()[['model_uuid']].T

In [ ]:
collections=['DILIst_txptr_class']
responsecols = 'DILIst_Classification'
allmodels3=cmp.get_multitask_perf_from_tracker(collections[0], response_cols=responsecols, 
                                              expand_subsets='train,test,val', exhaustive=True)
allmodels3=allmodels3[allmodels3.prediction_type=='classification']
allmodels3=allmodels3[allmodels3.features.str.contains('txptr')]
allmodels3['model_run']='With bio data'
allmodels3=allmodels3.reset_index(drop=True)
print(allmodels3.best_valid_roc_auc_score.max())
print(allmodels3.shape)
allmodels3.groupby(by=['model_type', 'features', 'best_valid_num_compounds']).count()[['model_uuid']].T

In [ ]:
allmodels=pd.concat([allmodels1, allmodels2, allmodels3])
allmodels = allmodels[allmodels.descriptor_type!= 'mordred_sel']
allmodels.shape

### Best RF model +/- txptr

In [ ]:
model_uuid=allmodels[(allmodels.model_type=='RF') & (allmodels.model_run=='Scram bio data')].sort_values('best_valid_roc_auc_score', ascending=False).model_uuid.iloc[0]
collection_name = collection = mt.get_model_collection_by_uuid(model_uuid)
top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
model_uuid

In [ ]:
top_model['training_metrics'][1]#['prediction_results']['r2_score']

In [ ]:
model_uuid=allmodels[(allmodels.model_type=='RF') & (allmodels.model_run=='With bio data')].sort_values('best_valid_roc_auc_score', ascending=False).model_uuid.iloc[0]
collection_name = collection = mt.get_model_collection_by_uuid(model_uuid)
top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
model_uuid

In [ ]:
top_model['training_metrics'][1]#['prediction_results']['r2_score']

# Best NN model

In [ ]:
model_uuid=allmodels[(allmodels.model_type=='NN') & (allmodels.model_run=='Scram bio data')].sort_values('best_valid_roc_auc_score', ascending=False).model_uuid.iloc[0]
collection_name = mt.get_model_collection_by_uuid(model_uuid)
top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
model_uuid

In [ ]:
top_model['training_metrics'][1]#['prediction_results']['r2_score']

In [ ]:
model_uuid=allmodels[(allmodels.model_type=='NN') & (allmodels.model_run=='With bio data')].sort_values('best_valid_roc_auc_score', ascending=False).model_uuid.iloc[0]
collection_name = mt.get_model_collection_by_uuid(model_uuid)
top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
model_uuid

In [ ]:
top_model['training_metrics'][1]#['prediction_results']['r2_score']

# Best XG model

In [ ]:
model_uuid=allmodels[(allmodels.model_type=='xgboost') & (allmodels.model_run=='Scram bio data')].sort_values('best_valid_roc_auc_score', ascending=False).model_uuid.iloc[0]
collection_name = collection = mt.get_model_collection_by_uuid(model_uuid)
top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
model_uuid

In [ ]:
top_model['training_metrics'][1]#['prediction_results']['r2_score']

In [ ]:
model_uuid=allmodels[(allmodels.model_type=='xgboost') & (allmodels.model_run=='With bio data')].sort_values('best_valid_roc_auc_score', ascending=False).model_uuid.iloc[0]
collection_name = collection = mt.get_model_collection_by_uuid(model_uuid)
top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
model_uuid

In [ ]:
top_model#['training_metrics'][1]#['prediction_results']['r2_score']

# predict w/ best model

In [ ]:
import os
import sys
sys.path.append('%s/workspace/bitbucket_repos/data_science/code' % os.environ['HOME'])

from atomsci.ddm.pipeline import predict_from_model as pfm
import plotting_functions_misc_akp as pfma
from sklearn.metrics import roc_curve, auc, roc_auc_score, r2_score, precision_recall_curve, average_precision_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, matthews_corrcoef, cohen_kappa_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set_context('poster')
sns.set_style('white')

In [ ]:
# adapted from sklearn 0.22.0 package
def balanced_accuracy_score(conf_mat, *, sample_weight=None,
                            adjusted=False):
    """Compute the balanced accuracy.
    The balanced accuracy in binary and multiclass classification problems to
    deal with imbalanced datasets. It is defined as the average of recall
    obtained on each class.
    The best value is 1 and the worst value is 0 when ``adjusted=False``.
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        per_class = np.diag(conf_mat) / conf_mat.sum(axis=1)
    if np.any(np.isnan(per_class)):
        warnings.warn('y_pred contains classes not in y_true')
        per_class = per_class[~np.isnan(per_class)]
    score = np.mean(per_class)
    if adjusted:
        n_classes = len(per_class)
        chance = 1 / n_classes
        score -= chance
        score /= 1 - chance
    return score

In [ ]:
allmodels['best_valid_balanced_accuracy'] = [balanced_accuracy_score(np.array(x[0])) for x in allmodels.best_valid_confusion_matrix]
allmodels['best_test_balanced_accuracy'] = [balanced_accuracy_score(np.array(x[0])) for x in allmodels.best_test_confusion_matrix]
allmodels['best_train_balanced_accuracy'] = [balanced_accuracy_score(np.array(x[0])) for x in allmodels.best_train_confusion_matrix]

In [ ]:
selection_metric = 'best_valid_roc_auc_score'
# selection_metric = 'best_valid_balanced_accuracy'
# selection_metric = 'best_valid_prc_auc_score'
# selection_metric = 'best_valid_accuracy_score'

In [ ]:
print('Best class scramble model', allmodels[allmodels.model_run=='Scram bio data'][selection_metric].max())
print('Best classification model', allmodels[allmodels.model_run=='No bio data'][selection_metric].max())
print('Best class+tbiodata model', allmodels[allmodels.model_run=='With bio data'][selection_metric].max())

In [ ]:
top_model1=allmodels[allmodels.model_run=='Scram bio data'].sort_values(selection_metric, ascending=False).iloc[0]
top_model2=allmodels[allmodels.model_run=='With bio data'].sort_values(selection_metric, ascending=False).iloc[0:10]
# top_models=[(top_model1.model_uuid, 'best base model'), (top_model2.model_uuid, 'best +bio model')]
top_models=top_model2.model_uuid.tolist()
print(top_models)
print(top_model1[selection_metric])
print(top_model2[selection_metric])

In [ ]:
# Create an array with the colors you want to use
colors = [
    "#7682A4",
    "#A7DDD8",
    "#373C50",
    "#694691",
    "#BE2369",
    "#EB1E23",
    "#6EC8BE",
    "#FFC30F",
]
# Set your custom color palette
pal=sns.color_palette(colors)
sns.set_palette(pal)
sns.palplot(pal)

In [ ]:
import importlib
importlib.reload(pfma)
import tempfile
from atomsci.ddm.pipeline import parameter_parser as parse
from atomsci.ddm.pipeline import model_pipeline as mp

sns.set_context('poster')
sns.set_style("white")
tcpl='_tcpl';adj='_adj'
id_col='compound_id'
smiles_col='base_rdkit_smiles'
pos_label=1
class_names=['DILIneg', 'DILIpos']

model_label='Best bio data model'

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    for model_uuid in top_models:

# set variable names & get data
        collection_name = mt.get_model_collection_by_uuid(model_uuid)
        top_model=mt.get_full_metadata_by_uuid(model_uuid, collection_name)
        dataset_key=top_model['training_dataset']['dataset_key']
        dataset = dataset_key.split('/')[-1].replace('.csv','')
        df=pd.read_csv(dataset_key, index_col=0)
        response_cols = top_model['training_dataset']['response_cols']
        if len(response_cols)==1: response_col=response_cols[0]
        else: response_col=response_cols
        featurizer= top_model['model_parameters']['featurizer']
        cols=['compound_id', 'DILIst_Classification', 'CompoundName', 'vDILIConcern', 'SeverityClass']
        split_uuid=top_model['splitting_parameters']['split_uuid']
        if featurizer == 'computed_descriptors':
            featurized = True
            feat_type=top_model['descriptor_specific']['descriptor_type']
            feat = pd.read_csv(f'/g/g16/apaulson/workspace/datasets/scaled_descriptors/{dataset}_with_{feat_type}_descriptors.csv')
        else:
            featurized = False
            feat_type = featurizer
            feat=df
        feat[id_col]=feat[id_col].astype(str)
        meta=df[cols]
        meta[id_col]=meta[id_col].astype(str)
        splitdf=pd.read_csv(f'/g/g16/apaulson/workspace/datasets/{dataset}_train_valid_test_scaffold_{split_uuid}.csv')
        splitdf['cmpd_id']=splitdf['cmpd_id'].astype(str)
        
# list model metrics from model tracker
        print('Training metrics from model tracker\n\n', top_model['training_metrics'][0]['prediction_results'], '\n')
        print('Validation metrics from model tracker\n\n', top_model['training_metrics'][1]['prediction_results'], '\n')
        print('Test metrics from model tracker\n\n', top_model['training_metrics'][2]['prediction_results'], '\n')
        
# predict (either predict from model or this code snippet give same results)
        pred_params = {
        'featurizer': 'computed_descriptors',
        'result_dir': tempfile.mkdtemp(),
        'id_col': id_col,
        'smiles_col': smiles_col,
        'response_cols': response_cols
        }
        pred_params = parse.wrapper(pred_params)
        # pipe, pparams = mp.load_from_tracker(model_uuid)
        pipe = mp.create_prediction_pipeline(pred_params, model_uuid, collection_name)
        if pipe.params.model_type == 'xgboost':
            pipe.params.uncertainty = False
        full_df = pipe.predict_full_dataset(feat, contains_responses=True, is_featurized=featurized,
                                            dset_params=pred_params)
#         full_df = pfm.predict_from_tracker_model(model_uuid, collection_name, feat, id_col,
#                                                  smiles_col, response_cols,
#                                                  is_featurized=featurized, dont_standardize=True)
        full_df=full_df.merge(meta, how="inner", on="compound_id")
        full_df=full_df.merge(splitdf, how="inner", left_on="compound_id", right_on="cmpd_id")
        full_df=full_df[~full_df[response_col].isna()]
        
# graph
        fig, ax = plt.subplots(2,2, sharex=False, sharey=False, figsize=(20,20))
        ax=ax.ravel()
       
    
# training set    
        pred_df=full_df[full_df.subset=='train']
        
    # calculate further metrics
        print('\nCalculated metrics from predictions')
        
        accscore = accuracy_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('\ntrain accuracy', accscore)
        
        cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('train CM', cm)
                
        average_precision = average_precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        print('train prc_auc', average_precision)
        
        prescore = precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("train precision", prescore)
        
        recscore =    recall_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("train recall", recscore)

        roc_auc = roc_auc_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("train roc_auc", roc_auc, '\n')
    # plot curve
        fpr, tpr, _ = roc_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        lw = 2
        ax[0].plot(fpr, tpr, lw=lw, label='Train ROC curve (area = %0.2f)' % roc_auc, color = pal[3])
    # prc 
        precision, recall, _ = precision_recall_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        line_kwargs = {"drawstyle": "steps-post"}
        line_kwargs["label"] = (f"AP = "
                                f"{average_precision:0.2f}")
        ax[1].plot(recall, precision, color=pal[3], **line_kwargs) 
        
# validation set
        pred_df=full_df[full_df.subset=='valid']
    
    # calculate further metrics
        accscore = accuracy_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('valid accuracy', accscore)
        
        cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('valid CM', cm)
        
        average_precision = average_precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        print('valid prc_auc', average_precision)
        
        prescore = precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("valid precision", prescore)
        
        recscore =    recall_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("valid recall", recscore)
        
        roc_auc = roc_auc_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("valid roc_auc", roc_auc, '\n')
        
    # ROC AUC curve
        fpr, tpr, _ = roc_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        lw = 2
        line_kwargs = {"drawstyle": "steps-post"}
        ax[0].plot(fpr, tpr, lw=lw, label='Valid ROC curve (area = %0.2f)' % roc_auc, color = pal[4], **line_kwargs)
#         ax[0].plot([0, 1], [0, 1], lw=lw, linestyle='--')
#         ax[0].set_xlim([0.0, 1.0])
#         ax[0].set_ylim([0.0, 1.05])
#         ax[0].set_xlabel('False Positive Rate')
#         ax[0].set_ylabel('True Positive Rate')
#         ax[0].legend(loc="lower right")
        
    # PR curve
        precision, recall, _ = precision_recall_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        line_kwargs = {"drawstyle": "steps-post"}
        line_kwargs["label"] = (f"AP = "
                                f"{average_precision:0.2f}")
        ax[1].plot(recall, precision, color=pal[4], **line_kwargs)
#         info_pos_label = (f" (Positive label: {pos_label})")
#         xlabel = "Recall" + info_pos_label
#         ylabel = "Precision" + info_pos_label
#         ax[1].set(xlabel=xlabel, ylabel=ylabel)
#         ax[1].set_ylim([-0.05, 1.05])
#         ax[1].set_ylim([-0.05, 1.05])
#         ax[1].legend(loc="upper right")
#         ax[1].set_title(f'{dataset} {model_label} validation set\n{model_uuid}')
        
    # confusion matrix - valid set (recalculated, not from table)
        cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('Valid balanced accuracy\n', balanced_accuracy_score(cm))
        # cm=np.array(top_model['training_metrics'][1]['prediction_results']['confusion_matrix'][0])
        im = pfma.plot_confusion_matrix(cm, classes=class_names, normalize=False, cmap=sns.cubehelix_palette(rot=0, start=2.40, as_cmap=True), ax=ax[2])
        fig.colorbar(mappable=im, ax=ax[2], shrink=0.7)
        ax[2].set_title("Valid CM")
        
        
# test set    
        pred_df=full_df[full_df.subset=='test']
    
    # calculate further metrics
        accscore = accuracy_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('test accuracy', accscore)
        
        cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('test CM', cm)
        
        average_precision = average_precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        print('test prc_auc', average_precision) 
        
        prescore = precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("test precision", prescore)
        
        recscore =    recall_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("test recall", recscore)
        
        roc_auc = roc_auc_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print("test roc_auc", roc_auc, '\n')
        
    # ROC AUC curve
        fpr, tpr, _ = roc_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        lw = 2
        line_kwargs = {"drawstyle": "steps-post"}
        ax[0].plot(fpr, tpr, lw=lw, label='Test ROC curve (area = %0.2f)' % roc_auc, color = pal[5], **line_kwargs)
        ax[0].plot([0, 1], [0, 1], lw=lw, linestyle='--')
        ax[0].set_xlim([-0.05, 1.05])
        ax[0].set_ylim([-0.05, 1.05])
        ax[0].set_xlabel('False Positive Rate')
        ax[0].set_ylabel('True Positive Rate')
        ax[0].legend(loc="lower right")
        
    # PR curve
        precision, recall, _ = precision_recall_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
        line_kwargs = {"drawstyle": "steps-post"}
        line_kwargs["label"] = (f"AP = "
                                f"{average_precision:0.2f}")
        ax[1].plot(recall, precision, color=pal[5], **line_kwargs)
        info_pos_label = (f" (Positive label: {pos_label})")
        xlabel = "Recall" + info_pos_label
        ylabel = "Precision" + info_pos_label
        ax[1].set(xlabel=xlabel, ylabel=ylabel)
        ax[1].set_xlim([-0.05, 1.05])
        ax[1].set_ylim([-0.05, 1.05])
        ax[1].legend(loc="lower right")
#         ax[1].set_title(f'{dataset} {model_label} test set\n{model_uuid}')
        
    # confusion matrix - test set  (recalculated, not from table)
        cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
        print('Test balanced accuracy\n', balanced_accuracy_score(cm))
        # cm=np.array(top_model['training_metrics'][2]['prediction_results']['confusion_matrix'][0])
        im=pfma.plot_confusion_matrix(cm, classes=class_names, normalize=False, cmap=sns.cubehelix_palette(rot=0, start=2.40, as_cmap=True), ax=ax[3])
        fig.colorbar(mappable=im, ax=ax[3], shrink=0.7)
        ax[3].set_title("Train CM")
#         plt.tight_layout()
        fig.suptitle(f'{dataset} {model_label}')#\n{model_uuid}')

In [ ]:
# pipe=mp.retrain_model('a7e54dec-a8aa-4d6d-a875-b15d29e464e7')

In [ ]:
# import atomsci.ddm.pipeline.perf_plots as pp

In [ ]:
# pp.plot_ROC_curve(pipe)

# examine hyperparams

In [ ]:
scoretype = 'balanced_accuracy'
winnertype= f'best_valid_{scoretype}'

perf_track_df=allmodels.sort_values(by=winnertype, ascending=False)

if (sum(~perf_track_df.dropouts.isna())>0):
    perf_track_df[['drop1', 'drop2', 'drop3']] = perf_track_df.dropouts.astype(str).str.strip('[]').str.split(pat=',',n=3, expand=True).astype(float)
    perf_track_df['layer_sizes'] = perf_track_df.layer_sizes.astype(str).str.strip('[]')
    perf_track_df[['layer1','layer2','layer3']]=perf_track_df.layer_sizes.str.split(pat=',', n=3, expand=True).astype(float)
    perf_track_df['num_layers'] = 3-perf_track_df[['layer1','layer2','layer3']].isna().sum(axis=1)
    perf_track_df[['layer1','layer2','layer3']]=perf_track_df[['layer1','layer2','layer3']].fillna(value=1).astype(int)
    perf_track_df['num_nodes']=perf_track_df.layer1 * perf_track_df.layer2 * perf_track_df.layer3
    perf_track_df.loc[perf_track_df.model_type != "NN", 'layer_sizes']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'num_layers']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'num_nodes']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'drop1']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'drop2']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'drop3']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'layer1']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'layer2']=np.nan
    perf_track_df.loc[perf_track_df.model_type != "NN", 'layer3']=np.nan

print(perf_track_df.shape)
# print(perf_track_df.columns)

print('top bal acc score:', perf_track_df[f'{winnertype}'].max().round(4))

scoretype = 'roc_auc_score'
winnertype= f'best_valid_{scoretype}'
print('top roc score:', perf_track_df[f'{winnertype}'].max().round(4))

scoretype = 'prc_auc_score'
winnertype= f'best_valid_{scoretype}'
print('top prc score:', perf_track_df[f'{winnertype}'].max().round(4))

In [ ]:
hyperparams = ['features', 'model_type', 'learning_rate','layer1', 'layer2', 'layer3', 'drop1', 'drop2', 'drop3',  'rf_estimators', 'rf_max_depth', 'rf_max_features','xgb_learning_rate', 'xgb_gamma', 'num_nodes', ]
loghps=['learning_rate', 'xgb_learning_rate']
len(hyperparams)

In [ ]:
fig, axes = plt.subplots(5,3,figsize=(30,20))
for i, ax in enumerate(axes.flat):
    plot = perf_track_df[hyperparams[i]]
    if isinstance(plot.iloc[0], str):
        g=sns.countplot(plot, ax=ax)
        if hyperparams[i]=='features':
            g.set_xticklabels(g.get_xticklabels(), rotation=20, ha='right')
    elif hyperparams[i] in loghps:
        sns.distplot(plot, hist_kws={'log':True}, ax=ax)
    else:
        sns.distplot(plot, ax=ax)
plt.tight_layout()

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
selection_metric = 'best_valid_roc_auc_score'
perf_track_df['txptr_features']=perf_track_df['features'].str.replace('_txptr', '').str.replace('_scr','')
perf_track_df=perf_track_df.sort_values(['model_run','features'])
plot_df=perf_track_df[(perf_track_df.txptr_features!='ecfp')&(perf_track_df.txptr_features!='graphconv')&(perf_track_df.model_run!='No bio data')]#&(perf_track_df.txptr_features!='mordred_seltrain')]
sns.set_style("ticks")
sns.catplot(x="txptr_features", y=selection_metric, 
            hue='model_run', height = 10, aspect=1, 
            kind = 'box',  showfliers = False,
            #col='model_type',
            data=plot_df);

plot_df=plot_df.reset_index(drop=True)
print(plot_df.shape)
scores = plot_df.pivot(columns='features', values=selection_metric)
print(scores.shape)
print(scores.max())
print("MOE",ttest_ind(scores.moe_scr, scores.moe_txptr, nan_policy='omit'))
print("MRS",ttest_ind(scores.mordred_filtered_scr, scores.mordred_filtered_txptr, nan_policy='omit'))
print("MRF",ttest_ind(scores.mordred_seltrain_scr, scores.mordred_seltrain_txptr, nan_policy='omit'))
print("RDK",ttest_ind(scores.rdkit_raw_scr, scores.rdkit_raw_txptr, nan_policy='omit'))

In [ ]:
scores

In [ ]:
scoretype='balanced_accuracy'
subset='best_valid'
winnertype= f'{subset}_{scoretype}'
plot_df=perf_track_df#[perf_track_df.descriptor_type=='mordred_seltrain']
plot_df=plot_df[[f"best_train_{scoretype}",f"best_valid_{scoretype}",f"best_test_{scoretype}"]]
# turn off sorting if you have a ton of models.. slow
plot_df=plot_df.sort_values(f"best_test_{scoretype}")

fig, ax = plt.subplots(1,2,figsize=(26,8))
sns.kdeplot(perf_track_df[f'best_train_{scoretype}'], label="train",ax=ax[0])
sns.kdeplot(perf_track_df[f'best_valid_{scoretype}'], label="valid",ax=ax[0])
sns.kdeplot(perf_track_df[f'best_test_{scoretype}'], label="test",ax=ax[0])

ax[0].legend(loc="upper left")
ax[1].plot(plot_df.T);
ax[1].set_xticklabels(['train','valid','test'])
ax[1].set_ylim(plot_df.min().min()-.1,1)
fig.suptitle(f"{scoretype} for DILIst classification models");

In [ ]:
# bin each hp for downstream viz
pd.set_option('display.float_format', '{:.2g}'.format)
# hyperparams.append('best_valid_balanced_accuracy')
hpbins =      [10,10,10,10,10,10,10,10,10,10,5,5,10,10]

for i, hp in enumerate(hyperparams[2:]):
    perf_track_df[['binned_'+hp+'_0', 'binned_'+hp]] = pd.cut(perf_track_df[hp], hpbins[i], precision=2).astype(str).str.strip('(]').str.split(',', expand=True)
    perf_track_df=perf_track_df.drop('binned_'+hp+'_0', axis='columns')
    perf_track_df['binned_'+hp] = perf_track_df['binned_'+hp].astype(float)

In [ ]:
# perf_track_df.binned_num_nodes=perf_track_df.binned_num_nodes.astype(float)
# perf_track_df.binned_num_nodes

In [ ]:
dataset_name='DILIst Classification'
if sum(~perf_track_df.rf_estimators.isna())>0:
    feat1 = 'binned_rf_estimators'; feat2 = 'binned_rf_max_depth'; feat3 = 'binned_rf_max_features'
    hue=feat3
    plot_df=perf_track_df[perf_track_df.model_type=='RF']
    plot_df = plot_df.sort_values([feat3, feat1, feat2])
    plot_df[f'{feat1}/{feat2}'] = ['%s / %s' % (mf,est) for mf,est in zip(plot_df[feat1], plot_df[feat2])]
    with sns.axes_style("whitegrid"):
        fig = plt.figure(figsize=(80,15))
        ax1 = fig.add_subplot(111)
        sns.scatterplot(x=f'{feat1}/{feat2}', y='{0}_{1}'.format(subset, scoretype), hue=hue, palette=sns.cubehelix_palette(len(plot_df[hue].unique())), data=plot_df, ax=ax1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xticks(rotation=30, ha='right')
        plt.title(f'%s RF model performance' % dataset_name);

In [ ]:
if sum(~perf_track_df.dropouts.isna())>0:
    feat1 = 'binned_learning_rate'; feat3 = 'binned_drop1'; feat2 = 'binned_layer1'
    hue=feat3
    plot_df=perf_track_df[perf_track_df.model_type=='NN']
    plot_df = plot_df.sort_values([feat3, feat1, feat2])
    plot_df[f'{feat1}/{feat2}'] = ['%s / %s' % (mf,est) for mf,est in zip(plot_df[feat1], plot_df[feat2])]
    with sns.axes_style("whitegrid"):
        fig = plt.figure(figsize=(80,15))
        ax1 = fig.add_subplot(111)
        sns.scatterplot(x=f'{feat1}/{feat2}', y='{0}_{1}'.format(subset, scoretype), hue=hue, palette=sns.cubehelix_palette(len(plot_df[hue].unique())), data=plot_df, ax=ax1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xticks(rotation=30, ha='right')
        plt.title(f'%s NN model performance - layer 1' % dataset_name);

In [ ]:
if sum(~perf_track_df.dropouts.isna())>0:
    feat1 = 'binned_learning_rate'; feat3 = 'binned_drop2'; feat2 = 'binned_layer2'
    hue=feat3
    plot_df=perf_track_df[perf_track_df.model_type=='NN']
    plot_df = plot_df.sort_values([feat3, feat1, feat2])
    plot_df[f'{feat1}/{feat2}'] = ['%s / %s' % (mf,est) for mf,est in zip(plot_df[feat1], plot_df[feat2])]
    with sns.axes_style("whitegrid"):
        fig = plt.figure(figsize=(80,15))
        ax1 = fig.add_subplot(111)
        sns.scatterplot(x=f'{feat1}/{feat2}', y='{0}_{1}'.format(subset, scoretype), hue=hue, palette=sns.cubehelix_palette(len(plot_df[hue].unique())), data=plot_df, ax=ax1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xticks(rotation=30, ha='right')
        plt.title(f'%s NN model performance - layer 2' % dataset_name);

In [ ]:
if sum(~perf_track_df.dropouts.isna())>0:
    feat1 = 'binned_learning_rate'; feat3 = 'binned_drop3'; feat2 = 'binned_layer3'
    hue=feat3
    plot_df=perf_track_df[perf_track_df.model_type=='NN']
    plot_df = plot_df.sort_values([feat3, feat1, feat2])
    plot_df[f'{feat1}/{feat2}'] = ['%s / %s' % (mf,est) for mf,est in zip(plot_df[feat1], plot_df[feat2])]
    with sns.axes_style("whitegrid"):
        fig = plt.figure(figsize=(80,15))
        ax1 = fig.add_subplot(111)
        sns.scatterplot(x=f'{feat1}/{feat2}', y='{0}_{1}'.format(subset, scoretype), hue=hue, palette=sns.cubehelix_palette(len(plot_df[hue].unique())-1), data=plot_df, ax=ax1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xticks(rotation=30, ha='right')
        plt.title(f'%s NN model performance - layer 3' % dataset_name);

In [ ]:
if sum(~perf_track_df.xgb_gamma.isna())>0:
    feat2 = 'binned_xgb_learning_rate'; feat1 = 'xgb_gamma'
    hue=feat2
    plot_df=perf_track_df[perf_track_df.model_type=='xgboost']
    plot_df = plot_df.sort_values([feat1, feat2])
    #plot_df[f'{feat1}/{feat2}'] = ['%s / %s' % (mf,est) for mf,est in zip(plot_df[feat1], plot_df[feat2])]
    with sns.axes_style("whitegrid"):
        fig = plt.figure(figsize=(40,15))
        ax1 = fig.add_subplot(111)
        sns.scatterplot(x=feat1, y='{0}_{1}'.format(subset, scoretype), 
                        hue=hue, palette=sns.cubehelix_palette(len(plot_df[hue].unique())), 
                        data=plot_df, ax=ax1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xticks(rotation=30, ha='center')
        plt.title(f'%s XGboost model performance' % dataset_name);

In [ ]:
nfeats=3; scoretype='balanced_accuracy'; winnertype='best_valid_balanced_accuracy'
feat1='binned_drop1'; feat2='binned_learning_rate'; feat3='binned_num_nodes'; feat4='binned_layer1'
feat5='binned_rf_max_depth'; feat6='binned_rf_max_features'; feat7='binned_rf_estimators'; 
feat8='binned_xgb_gamma'; feat9='binned_xgb_learning_rate'; feat12='best_test_balanced_accuracy'
feat10='features'; feat11='model_type'; 

In [ ]:
import math
bnnlabs = perf_track_df.binned_num_nodes.sort_values(ascending=True).unique().tolist()
bnnlabs.pop(-1) #remove nan - comment out if only NN models in df
bnnlabs = [round(x, 3 - int(math.floor(math.log10(abs(x)))) - 1) for x in bnnlabs]
bnnlabs

In [ ]:
# ax[0,0].ticklabel_format(style='sci', scilimits=(0,0)); 
sns.set_context('poster')
plotdf2=perf_track_df
fig, ax = plt.subplots(3,4, figsize=(60,40))
if sum(~perf_track_df.dropouts.isna())>0:
    sns.boxplot(x=feat1, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat1].unique()), rot=0, start=0.60), data=plotdf2, ax=ax[0,0]); ax[0,0].tick_params(rotation=0); ax[0,0].set_xlabel('NN dropouts layer 1')
    sns.boxplot(x=feat2, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat2].unique()), rot=0, start=0.60), data=plotdf2, ax=ax[0,1]); ax[0,1].tick_params(rotation=0); ax[0,1].set_xlabel('NN learning rate')#ax[0,1].legend_.remove(); ax[0,1].title.set_text(f"Hyperparameters colored by {feat1}")
    sns.boxplot(x=feat3, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat3].unique()), rot=0, start=0.60), data=plotdf2, ax=ax[0,2]); ax[0,2].tick_params(rotation=15);ax[0,2].set_xlabel('NN number of parameters in hidden layers'); ax[0,2].set_xticklabels(bnnlabs); #ax[0,2].legend_.remove()#(bbox_to_anchor=(1,1), title=feat1)#, prop={'size': 12})
    sns.boxplot(x=feat4, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat4].unique()), rot=0, start=0.60), data=plotdf2, ax=ax[0,3]); ax[0,3].tick_params(rotation=0); ax[0,3].set_xlabel('NN params layer 1')#ax[1,0].legend_.remove(); ax[1,0].tick_params(rotation=45)
if sum(~perf_track_df.rf_estimators.isna())>0:
    sns.boxplot(x=feat5, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat5].unique()), rot=0, start=2.00), data=plotdf2, ax=ax[1,0]); ax[1,0].tick_params(rotation=0); ax[1,0].set_xlabel('RF max depth')#ax[1,1].title.set_text(f"Hyperparameters colored by {feat2}")
    sns.boxplot(x=feat6, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat6].unique()), rot=0, start=2.00), data=plotdf2, ax=ax[1,1]); ax[1,1].tick_params(rotation=0); ax[1,1].set_xlabel('RF max features per node')#ax[1,2].legend_.remove()#(bbox_to_anchor=(1,1), title=feat2)
    sns.boxplot(x=feat7, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat7].unique()), rot=0, start=2.00), data=plotdf2, ax=ax[1,2]); ax[1,2].tick_params(rotation=0); ax[1,2].set_xlabel('RF number of trees')#ax[2,0].legend_.remove(); ax[2,0].tick_params(rotation=45)
sns.scatterplot(x=feat12,y=winnertype, color=pal,                                                                      data=plotdf2, ax=ax[1,3]); ax[1,3].tick_params(rotation=0); ax[1,3].set_xlabel(f'{feat12}')#ax[2,1].legend_.remove(); ax[2,1].title.set_text(f"Hyperparameters colored by {feat3}")
if sum(~perf_track_df.xgb_gamma.isna())>0:
    sns.boxplot(x=feat8, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat8].unique()), rot=0, start=2.40), data=plotdf2, ax=ax[2,0]); ax[2,0].tick_params(rotation=0); ax[2,0].set_xlabel('XGBoost gamma')#ax[2,2].legend(bbox_to_anchor=(1,1), title=feat3);
    sns.boxplot(x=feat9, y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat9].unique()), rot=0, start=2.40), data=plotdf2, ax=ax[2,1]); ax[2,1].tick_params(rotation=0); ax[2,1].set_xlabel('XGBoost learning rate')#ax[2,0].legend_.remove(); ax[2,0].tick_params(rotation=45)
sns.boxplot(    x=feat10,y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat10].unique()),rot=0, start=0.10), data=plotdf2, ax=ax[2,2]); ax[2,2].tick_params(rotation=30);ax[2,2].set_xlabel('Featurization type')#ax[2,1].legend_.remove(); ax[2,1].title.set_text(f"Hyperparameters colored by {feat3}")
sns.boxplot(    x=feat11,y=winnertype, palette=sns.cubehelix_palette(len(plotdf2[feat11].unique()),rot=0, start=0.10), data=plotdf2, ax=ax[2,3]); ax[2,3].tick_params(rotation=0); ax[2,3].set_xlabel('Model type');#ax[2,2].legend(bbox_to_anchor=(1,1), title=feat3);
plt.tight_layout()